In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path

In [15]:
from dealDiscount.constants import *
from dealDiscount.utils.common import read_yaml, create_directories
class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
from dealDiscount import logger


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if os.path.exists(self.config.local_data_file):
            logger.info(f"Data already exists at {self.config.local_data_file}")
            return
        
        logger.info(f"Downloading data from {self.config.source_url}")
        request.urlretrieve(self.config.source_url, self.config.local_data_file)
        logger.info(f"Data downloaded at {self.config.local_data_file}")

        

In [16]:
# Pipeline step 1: Data Ingestion
try:
    config_mgr = ConfigurationManager()
    data_ingestion_config = config_mgr.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
except Exception as e:
    logger.error(e)
    raise e

[2023-06-06 17:28:57,540: INFO: common: Reading yaml file: config/config.yaml]
[2023-06-06 17:28:57,542: INFO: common: Reading yaml file: params.yaml]
[2023-06-06 17:28:57,543: INFO: common: Created directory: artifacts/data_ingestion]
[2023-06-06 17:28:57,544: INFO: 1595801065: Downloading data from https://github.com/sandeeppvn/Sales-Deal-Discount-Optimization/blob/main/Data/DealData.xlsx]
[2023-06-06 17:28:58,222: INFO: 1595801065: Data downloaded at artifacts/data_ingestion/DealData.xlsx]
